In [1]:
# pip install tensorflow numpy
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [2]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()


In [3]:
x_train = np.expand_dims(x_train, -1)
x_test  = np.expand_dims(x_test, -1)

In [4]:
# Ensure 'tf' is defined by running the import cell (BklCHBJ18IBL) first.
x_train = tf.image.resize(x_train, [32, 32])
x_test  = tf.image.resize(x_test, [32, 32])

In [5]:

x_train = tf.image.grayscale_to_rgb(x_train)
x_test  = tf.image.grayscale_to_rgb(x_test)

In [6]:

x_train = x_train / 255.0
x_test  = x_test / 255.0

In [7]:
y_train = keras.utils.to_categorical(y_train, 10)
y_test  = keras.utils.to_categorical(y_test, 10)


In [8]:
base_model = keras.applications.MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(32, 32, 3)
)

base_model.trainable = False  # Freeze pretrained layers

C:\Users\Mayank\AppData\Local\Temp\ipykernel_20228\1882132568.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = keras.applications.MobileNetV2(


In [9]:
model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')   # MNIST has 10 classes
])

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 1, 1, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,423,242 (9.24 MB)

 Trainable params: 165,258 (645.54 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [10]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [11]:
# Ensure model is defined and compiled by running preceding cells first.
history = model.fit(
    x_train, y_train,
    epochs=3,
    batch_size=64,
    validation_split=0.1
)

Epoch 1/3


844/844 ━━━━━━━━━━━━━━━━━━━━ 27s 27ms/step - accuracy: 0.6162 - loss: 1.1731 - val_accuracy: 0.6880 - val_loss: 0.9220
Epoch 2/3
844/844 ━━━━━━━━━━━━━━━━━━━━ 28s 33ms/step - accuracy: 0.6813 - loss: 0.9383 - val_accuracy: 0.7038 - val_loss: 0.8704
Epoch 3/3
844/844 ━━━━━━━━━━━━━━━━━━━━ 26s 31ms/step - accuracy: 0.6953 - loss: 0.8948 - val_accuracy: 0.7117 - val_loss: 0.8352


In [12]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test Accuracy:", test_acc)


313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.6987 - loss: 0.8813
Test Accuracy: 0.6987000107765198
